In [1]:
%%capture
import kagglehub

# Download latest version
path = kagglehub.model_download("trainld/qwen2.5-14/pyTorch/default")

In [2]:
print("Path to model files:", path)

Path to model files: /root/.cache/kagglehub/models/trainld/qwen2.5-14/pyTorch/default/1


In [3]:
%%capture
import kagglehub

# Download latest version
path = kagglehub.model_download("dipamc77/sfr-embedding-mistral/transformers/default")

In [4]:
print("Path to model files:", path)

Path to model files: /root/.cache/kagglehub/models/dipamc77/sfr-embedding-mistral/transformers/default/1


In [5]:
!pip install opendatasets

In [6]:
import opendatasets as od

od.download("https://www.kaggle.com/datasets/sayoulala/bitsandbytes0-42-0")
od.download("https://www.kaggle.com/datasets/leonshangguan/eedi-sythetic-data")
od.download("https://www.kaggle.com/datasets/emiz6413/lmsys-wheel-files")
od.download("https://www.kaggle.com/datasets/zuoyouzuo/qwen14b-it-lora")
od.download("https://www.kaggle.com/datasets/ragnar123/qwen2-5-7b")
od.download("https://www.kaggle.com/datasets/ragnar123/qwen2-5-7b-instruct")
od.download("https://www.kaggle.com/datasets/sayoulala/sfr-embedding-mistral")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: syedosamaalishah092
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/sayoulala/bitsandbytes0-42-0


100%|██████████| 2.91G/2.91G [00:26<00:00, 119MB/s]



Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: syedosamaalishah092
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/leonshangguan/eedi-sythetic-data


100%|██████████| 26.0k/26.0k [00:00<00:00, 13.6MB/s]


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:

...

100%|██████████| 11.3G/11.3G [01:30<00:00, 134MB/s]



Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: syedosamaalishah092
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/ragnar123/qwen2-5-7b-instruct


100%|██████████| 11.3G/11.3G [01:28<00:00, 137MB/s] 



Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: syedosamaalishah092
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/sayoulala/sfr-embedding-mistral


100%|██████████| 10.5G/10.5G [01:24<00:00, 133MB/s]


In [7]:
!pip install transformers peft accelerate \
    -U --no-index --find-links /content/lmsys-wheel-files
!pip install --no-index /content/bitsandbytes0-42-0/bitsandbytes-0.42.0-py3-none-any.whl --find-links=/content/bitsandbytes0-42-0

Looking in links: /content/lmsys-wheel-files
Looking in links: /content/bitsandbytes0-42-0
Processing ./bitsandbytes0-42-0/bitsandbytes-0.42.0-py3-none-any.whl


In [8]:
import torch
print(torch.cuda.is_available())

True


In [9]:
from tqdm.auto import tqdm
from bs4 import BeautifulSoup
import gc
import pandas as pd
import pickle
import sys
import numpy as np
from tqdm.autonotebook import trange
from sklearn.model_selection import GroupKFold
import json
import torch
from numpy.linalg import norm
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel,BitsAndBytesConfig
from peft import (
    LoraConfig,
    get_peft_model,
)
import json
import copy
import warnings
import os
warnings.filterwarnings('ignore')

def apk(actual, predicted, k=25):
    """
    Computes the average precision at k.

    This function computes the average prescision at k between two lists of
    items.

    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The average precision at k over the input lists
    """

    if not actual:
        return 0.0

    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    return score / min(len(actual), k)

def mapk(actual, predicted, k=25):
    """
    Computes the mean average precision at k.

    This function computes the mean average prescision at k between two lists
    of lists of items.

    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """

    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

def batch_to_device(batch, target_device):
    """
    send a pytorch batch to a device (CPU/GPU)
    """
    for key in batch:
        if isinstance(batch[key], Tensor):
            batch[key] = batch[key].to(target_device)
    return batch

def last_token_pool(last_hidden_states: Tensor,
                    attention_mask: Tensor) -> Tensor:
    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]

def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'Instruct: {task_description}\nQuery: {query}'

def inference(df, model, tokenizer, device):
    batch_size = 16
    max_length = 512
    sentences = list(df['query_text'].values)
    pids = list(df['order_index'].values)
    all_embeddings = []
    length_sorted_idx = np.argsort([-len(sen) for sen in sentences])
    sentences_sorted = [sentences[idx] for idx in length_sorted_idx]
    for start_index in trange(0, len(sentences), batch_size, desc="Batches", disable=False):
        sentences_batch = sentences_sorted[start_index: start_index + batch_size]
        features = tokenizer(sentences_batch, max_length=max_length, padding=True, truncation=True,
                             return_tensors="pt")
        features = batch_to_device(features, device)
        with torch.no_grad():
            outputs = model(**features)
            embeddings = last_token_pool(outputs.last_hidden_state, features['attention_mask'])
            embeddings = torch.nn.functional.normalize(embeddings, dim=-1)
            embeddings = embeddings.detach().cpu().numpy().tolist()
        all_embeddings.extend(embeddings)

    all_embeddings = [np.array(all_embeddings[idx]).reshape(1, -1) for idx in np.argsort(length_sorted_idx)]

    sentence_embeddings = np.concatenate(all_embeddings, axis=0)
    result = {pids[i]: em for i, em in enumerate(sentence_embeddings)}
    return result

In [10]:
path_prefix = "/content/eedi-mining-misconceptions-in-mathematics"
model_path = "/root/.cache/kagglehub/models/trainld/qwen2.5-14/pyTorch/default/1"

lora_path='/content/qwen14b-it-lora/lora_weights/adapter.bin'
device='cuda:0'
VALID = False

In [11]:
tokenizer = AutoTokenizer.from_pretrained(lora_path.replace("/adapter.bin",""))
bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16
        )
model = AutoModel.from_pretrained(model_path,
                                  quantization_config=bnb_config,
                                  device_map=device,
                                  trust_remote_code=True)

if lora_path:
    print("loading lora")
    config = LoraConfig(
        r=64,
        lora_alpha=128,
        target_modules=[
            "q_proj",
            "k_proj",
            "v_proj",
            "o_proj",
            "gate_proj",
            "up_proj",
            "down_proj",
        ],
        bias="none",
        lora_dropout=0.05,
        task_type="FEATURE_EXTRACTION",
    )
    model = get_peft_model(model, config)
    d = torch.load(lora_path, map_location=model.device)
    model.load_state_dict(d, strict=False)
    model = model.merge_and_unload()
model = model.eval()
# model = model.to(device)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

loading lora


In [12]:
task_description = 'Given a math question with correct answer and a misconcepted incorrect answer, retrieve the most accurate misconception for the incorrect answer.'

In [13]:
tra = pd.read_csv("/content/eedi-sythetic-data/eedi_sythetic.csv")
print(tra.shape)

misconception_mapping = pd.read_csv(f"{path_prefix}/misconception_mapping.csv")

(1706, 7)


In [14]:
train_data = []
for _,row in tra.iterrows():
    real_text = row['CorrectAnswerText']
    query_text = f"### SubjectName: {row['SubjectName']}\n### ConstructName: {row['ConstructName']}\n### Question: {row['QuestionText']}\n### Correct Answer: {real_text}\n### Misconcepte Incorrect answer: {row['AnswerText']}"
    row['query_text'] = get_detailed_instruct(task_description, query_text)
    # row['answer_name'] = c
    train_data.append(copy.deepcopy(row))
train_df = pd.DataFrame(train_data)
train_df['order_index'] = list(range(len(train_df)))

In [15]:
train_embeddings = inference(train_df, model, tokenizer, device)

Batches:   0%|          | 0/107 [00:00<?, ?it/s]

In [16]:
misconception_mapping['query_text'] = misconception_mapping['MisconceptionName']
misconception_mapping['order_index'] = misconception_mapping['MisconceptionId']
doc_embeddings = inference(misconception_mapping, model, tokenizer, device)

Batches:   0%|          | 0/162 [00:00<?, ?it/s]

In [17]:
sentence_embeddings = np.concatenate([e.reshape(1, -1) for e in list(doc_embeddings.values())])
index_text_embeddings_index = {index: paper_id for index, paper_id in
                                         enumerate(list(doc_embeddings.keys()))}

In [18]:
predicts_test = []
for _, row in tqdm(train_df.iterrows()):
    query_id = row['order_index']
    query_em = train_embeddings[query_id].reshape(1, -1)

    cosine_similarity = np.dot(query_em, sentence_embeddings.T).flatten()

    sort_index = np.argsort(-cosine_similarity)[:25]
    pids = [index_text_embeddings_index[index] for index in sort_index]
    predicts_test.append(pids)

0it [00:00, ?it/s]

In [19]:
gt = [[train_df.iloc[i]['MisconceptionID']] for i in range(len(train_df))]
pr = predicts_test
print("Testing result MAPK =", mapk(gt, pr))

Testing result MAPK = 0.46006586250767073
